<a href="https://colab.research.google.com/github/irinagetman1973/Super-Resolution/blob/Holograms/Holograms_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this block to mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing libraries
import numpy as np
import math
from PIL import Image    
import cv2
import os
import random 
import glob
import sys
import tensorflow as tf


# Generating High Resolution Holograms

In [ ]:
filepath='/content/drive/MyDrive/Testing/train_HR'

samples=100
for n in range(1,samples+1):
    # Number of pixels
    S=1024;
    I=np.zeros((S,S),dtype=np.float64)

    # Implement a function that randomises the size , rotation and relative position of the object of the centre by +- 10%
    # we need more objects later
    for row in range(510,510+random.randrange(1,5)): # Randomize width between 1 and 5 pixels 
        for col in range(510,510+random.randrange(1,5)): # Randomize length between 1 and 5 pixels 
            I[row, col]=1

    # Parameter setup
    M = len(I)
    deltax=1.1*10**(-6) #pixel pitch 0.001 cm (10 um)
    w=405*10**(-9); # wavelength 405 nm
    k0 = 2*math.pi/w

    # implement a function that adjust this value by +- 10%
    z=0.0015/1000*random.choice([random.randrange(-1100,-1001,1),random.randrange(1000,1101,1)])  ; #25 cm, propagation distance

    # Step 1: simulation of propagation using the ASM
    deltaf=1/M/deltax

    x = list(range(1, M+1))
    C, R= np.meshgrid(x, x)

    # FORWARD PROPAGATION USING ASM
    Ref = 1*math.sqrt(np.mean(I))*np.exp(-1j*k0*z) #Reference wave / 50 -> 1
    A01=np.fft.fftshift(np.fft.ifft2(np.fft.fftshift(I))); #Spectrum first object

    p1=np.exp(-2j*math.pi*z*np.sqrt((1/w)**2-((R-M/2-1)*deltaf)**2-((C-M/2-1)*deltaf)**2))

    Az1=A01*p1#Propagating

    H1 = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(Az1)))

    # H1 = np.dot((abs(H1 + Ref)),(abs(H1 + Ref)))
    H1 = (abs(H1 + Ref))**2

    H1_=abs(H1)-np.amin(abs(H1))
    img1 = Image.fromarray(np.uint8((abs(H1_)/np.amax(abs(H1_))) * 255),'L')
    # img1.save(f"{filepath}/H1_{n}.png")
    filename = str(n).zfill(5)
    img1.save(f"{filepath}/{filename}.png")
    print('progressing, image',n,'/',samples)
    # width,height=H1.shape
    # img1.thumbnail((width/2,height/2), Image.ANTIALIAS)
    # img1.show()
    # img1.save(f"{filepath}/H2.png")

# Generating low resolution images

In [ ]:
from PIL import Image
import math
folder='/content/drive/MyDrive/Testing/train_HR'
filepath='/content/drive/MyDrive/Testing/train_LR/X4'
filenames=os.listdir(folder)
filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f)))) # Sort files in ascending order

samples=100
n=0

for filename in filenames:
  im = Image.open(os.path.join(folder,filename))
  im=im.resize((512, 512), Image.ANTIALIAS)

  new_filename=os.path.splitext(filename)[0]
  im.save(f"{filepath}/{new_filename}.png", format='PNG')
  n=n+1

  # if (n%500==0):
  print('progressing, image',n,'/',samples)

  if (n==samples):
    break

progressing, image 1 / 100
progressing, image 2 / 100
progressing, image 3 / 100
progressing, image 4 / 100
progressing, image 5 / 100
progressing, image 6 / 100
progressing, image 7 / 100
progressing, image 8 / 100
progressing, image 9 / 100
progressing, image 10 / 100
progressing, image 11 / 100
progressing, image 12 / 100
progressing, image 13 / 100
progressing, image 14 / 100
progressing, image 15 / 100
progressing, image 16 / 100
progressing, image 17 / 100
progressing, image 18 / 100
progressing, image 19 / 100
progressing, image 20 / 100
progressing, image 21 / 100
progressing, image 22 / 100
progressing, image 23 / 100
progressing, image 24 / 100
progressing, image 25 / 100
progressing, image 26 / 100
progressing, image 27 / 100
progressing, image 28 / 100
progressing, image 29 / 100
progressing, image 30 / 100
progressing, image 31 / 100
progressing, image 32 / 100
progressing, image 33 / 100
progressing, image 34 / 100
progressing, image 35 / 100
progressing, image 36 / 100
p

# Comparing HR and LR side to side

In [ ]:
# Merging images together
from PIL import Image

def merge_images(file1, file2):
    """Merge two images into one, displayed side by side
    :param file1: path to first image file
    :param file2: path to second image file
    :return: the merged Image object
    """
    image1 = Image.open(file1)
    image2 = Image.open(file2)

    (width1, height1) = image1.size
    (width2, height2) = image2.size

    result_width = width1 + width2
    result_height = max(height1, height2)

    result = Image.new('RGB', (result_width, result_height))
    result.paste(im=image1, box=(0, 0))
    result.paste(im=image2, box=(width1, 0))
    return result

hr_filepath='/content/drive/MyDrive/Testing/train_HR'
lp_filepath='/content/drive/MyDrive/Testing/train_LR/X4'
hr_filenames=os.listdir(hr_filepath)
hr_filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f)))) # Sort files in ascending order
lp_filenames=os.listdir(lp_filepath)
lp_filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f)))) # Sort files in ascending order


samples=10

for n in range(0,samples):
  pic=merge_images(hr_filepath+'/'+hr_filenames[n], lp_filepath+'/'+lp_filenames[n])
  pic.save(f'/content/drive/MyDrive/Holograms/Compare/{n}.png')
  n=n+1
  print('progressing, image',n,'/',samples)

  if (n==samples):
    break


progressing, image 1 / 10
progressing, image 2 / 10
progressing, image 3 / 10
progressing, image 4 / 10
progressing, image 5 / 10
progressing, image 6 / 10
progressing, image 7 / 10
progressing, image 8 / 10
progressing, image 9 / 10
progressing, image 10 / 10
